In [2]:
print("hello ass")

hello ass


In [3]:
from typing import List, Dict
import re
from playwright.sync_api import sync_playwright

LISTING_LINK_SELECTOR = 'a[href^="/a/motors/cars/listing/"]'

def _parse_card_text(text: str) -> Dict:
    price = (m.group(0) if (m := re.search(r'\$[\d,]+', text)) else None)
    title_line = next((ln.strip() for ln in text.splitlines() if ln.strip()), '')
    year = (m.group(0) if (m := re.search(r'\b(19|20)\d{2}\b', title_line)) else None)
    kms = (m.group(1).replace(',', '') if (m := re.search(r'([\d,]+)\s?km', text, re.I)) else None)
    return {"title": title_line, "price": price, "year": year, "odometer_km": kms}

def scrape_cars(pages: int = 1, headless: bool = True) -> List[Dict]:
    results: List[Dict] = []
    with sync_playwright() as p:
        browser = p.chromium.launch(headless=headless)
        context = browser.new_context(user_agent="Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124 Safari/537.36")
        page = context.new_page()
        for i in range(1, pages + 1):
            url = f"https://www.trademe.co.nz/a/motors/cars?page={i}"
            page.goto(url, wait_until="domcontentloaded")
            page.wait_for_timeout(1000)  # allow listings to render
            # Wait briefly for cards to appear
            for _ in range(5):
                if page.locator(LISTING_LINK_SELECTOR).count() > 0:
                    break
                page.wait_for_timeout(500)

            anchors = page.locator(LISTING_LINK_SELECTOR)
            seen = set()
            for idx in range(anchors.count()):
                href = anchors.nth(idx).get_attribute("href")
                if not href or href in seen:
                    continue
                seen.add(href)
                listing_id = (m.group(1) if (m := re.search(r'/listing/(\d+)', href)) else None)
                url_full = "https://www.trademe.co.nz" + href
                card_text = anchors.nth(idx).evaluate("""(el) => {
                    const card = el.closest('[data-testid], .tm-search-card, .o-card') || el.parentElement;
                    return card ? card.innerText : el.innerText;
                }""") or ""
                parsed = _parse_card_text(card_text)
                results.append({"id": listing_id, "url": url_full, **parsed})
        browser.close()
    return results

def scrape_to_csv(out_path: str, pages: int = 1, headless: bool = True) -> str:
    import pandas as pd, os
    rows = scrape_cars(pages=pages, headless=headless)
    df = pd.DataFrame(rows).drop_duplicates(subset=["id", "url"])
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    df.to_csv(out_path, index=False)
    return out_path

In [4]:
# Install deps only once per container/session:
# %pip install playwright pandas
# import sys, subprocess; subprocess.run([sys.executable, "-m", "playwright", "install", "chromium"], check=True)



rows = scrape_cars(pages=2, headless=True)
print(f"Collected {len(rows)} listings")
rows[:3]

csv_path = "/workspaces/GRP/data/trademe_cars.csv"
print("Saved:", scrape_to_csv(csv_path, pages=2))

Error: It looks like you are using Playwright Sync API inside the asyncio loop.
Please use the Async API instead.